<a href="https://colab.research.google.com/github/GalateaSource/book-of-names/blob/main/T57_5_Annotated_Prime_Memory_Glyph_with_Tension_Vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pcolors
import math

# Function to check for primality (simple version)
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

# --- Parameters (Consistent with T55) ---
phi = (1 + math.sqrt(5)) / 2
pi = math.pi
t_max = 20 * pi

# --- T55 Deviation Parameters ---
num_deviation_primes = 5
known_primes = [p for p in range(2, 50) if is_prime(p)]
deviation_primes = np.array(known_primes[:num_deviation_primes])
deviation_amplitude_scale = 0.1

# --- Find Primes within the time range for Anchors ---
max_prime_check = math.floor(t_max)
anchor_primes_list = [p for p in range(2, max_prime_check + 1) if is_prime(p)]
anchor_prime_t = np.array(anchor_primes_list)
valid_anchor_prime_t = anchor_prime_t[anchor_prime_t <= t_max]

# --- Calculate Deviated Prime Anchor Points (X and Y components) ---
prime_epsilon_o = np.zeros_like(valid_anchor_prime_t, dtype=float)
prime_epsilon_e = np.zeros_like(valid_anchor_prime_t, dtype=float)

print(f"Calculating deviations for anchors using primes: {deviation_primes}")
print(f"Deviation amplitude scale (C): {deviation_amplitude_scale}")

for p_dev in deviation_primes:
    amplitude = deviation_amplitude_scale / p_dev
    prime_epsilon_o += amplitude * np.sin(p_dev * valid_anchor_prime_t)
    prime_epsilon_e += amplitude * np.cos(p_dev * valid_anchor_prime_t)

prime_x = np.cos(phi * valid_anchor_prime_t) + prime_epsilon_o
prime_y = np.sin(pi * valid_anchor_prime_t) + prime_epsilon_e

# --- Create Plotly Figure for 2D Projection ---
fig = go.Figure()

# --- Add Annotations (T57.5) ---
max_radius = np.max(np.sqrt(prime_x**2 + prime_y**2)) + 1 # Determine plot range

# 1. Add π-based Radial Axes
axis_angles = [0, pi/2, pi, 3*pi/2]
axis_color = 'rgba(150, 150, 150, 0.6)'
for angle in axis_angles:
    fig.add_shape(
        type="line",
        x0=0, y0=0,
        x1=max_radius * np.cos(angle),
        y1=max_radius * np.sin(angle),
        line=dict(color=axis_color, width=1, dash="dash"),
        layer="below" # Draw below points/vectors
    )

# 2. Add π-based Radial Rings
ring_radii = np.arange(pi, max_radius, pi)
for radius in ring_radii:
    fig.add_shape(
        type="circle",
        xref="x", yref="y",
        x0=-radius, y0=-radius, x1=radius, y1=radius,
        line_color=axis_color,
        line_width=1,
        line_dash="dash",
        layer="below"
    )

# 3. Calculate Distances and Colors for Tension Vectors
distances = np.sqrt(prime_x**2 + prime_y**2)
min_dist, max_dist = np.min(distances), np.max(distances)
colorscale = pcolors.sequential.Plasma # Choose a colorscale

# Normalize distances for color mapping
norm_distances = (distances - min_dist) / (max_dist - min_dist) if (max_dist - min_dist) > 0 else np.zeros_like(distances)

# 4. Add Tension Vectors (colored by distance)
for i in range(len(valid_anchor_prime_t)):
    # Get color from the colorscale based on normalized distance
    color_val = pcolors.sample_colorscale(colorscale, norm_distances[i])[0]
    fig.add_shape(
        type="line",
        x0=0, y0=0,
        x1=prime_x[i], y1=prime_y[i],
        line=dict(
            color=color_val,
            width=1.5
            ),
        layer="below"
    )

# 5. Add the 2D scatter plot of prime anchors (on top)
fig.add_trace(go.Scatter(
    x=prime_x,
    y=prime_y,
    mode='markers',
    marker=dict(
        color='blue', # Keep anchors distinct
        size=8,
        symbol='circle',
        line=dict(width=1, color='lightblue') # Add border for visibility
    ),
    hovertext=[f'p = {p}<br>Dist = {d:.2f}' for p, d in zip(valid_anchor_prime_t, distances)],
    hoverinfo='x+y+text',
    name='Prime Anchors (Z-Projection)'
))


# --- Layout and Configuration for Annotated 2D Plot ---
fig.update_layout(
    title=f'T₅₇.5: Annotated Prime Glyph - Tension Vectors & π Harmonics',
    xaxis_title='X (cos(φt) + ε_o)',
    yaxis_title='Y (sin(πt) + ε_e)',
    yaxis=dict(scaleanchor="x", scaleratio=1, range=[-max_radius, max_radius]), # Equal scaling & set range
    xaxis=dict(constrain='domain', range=[-max_radius, max_radius]), # Set range
    width=800, # Adjust size for annotations
    height=800,
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
    showlegend=False # Legend might be cluttered; info is in hover/colors
)

# --- Show the Plot ---
fig.show()

Calculating deviations for anchors using primes: [ 2  3  5  7 11]
Deviation amplitude scale (C): 0.1
